In [45]:
!conda install -c conda-forge xgboost --yes

^C

CondaError: KeyboardInterrupt



In [46]:
!conda install -c conda-forge category_encoders --yes

^C

CondaError: KeyboardInterrupt



In [ ]:
!conda install -c conda-forge eli5 --yes

In [28]:
!conda update scikit-learn --yes

Solving environment: done

# All requested packages already installed.



In [47]:
import pandas as pd
test_features = pd.read_csv('test_features.csv')
train_features = pd.read_csv('train_features.csv')
train_labels = pd.read_csv('train_labels.csv')

In [48]:
train_labels['status_group'].value_counts(normalize=True)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

In [49]:
pd.set_option('display.max_columns', 1000)
test_features.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,Internal,Magoma,Manyara,21,3,Mbulu,Bashay,321,True,GeoData Consultants Ltd,Parastatal,NaN,True,2012,other,other,other,parastatal,parastatal,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,Pangani,Kimnyak,Arusha,2,2,Arusha Rural,Kimnyaki,300,True,GeoData Consultants Ltd,VWC,TPRI pipe line,True,2000,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
2,17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,Internal,Msatu,Singida,13,2,Singida Rural,Puma,500,True,GeoData Consultants Ltd,VWC,P,NaN,2010,other,other,other,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
3,45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,Ruvuma / Southern Coast,Kipindimbi,Lindi,80,43,Liwale,Mkutano,250,NaN,GeoData Consultants Ltd,VWC,NaN,True,1987,other,other,other,vwc,user-group,unknown,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
4,49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,Ruvuma / Southern Coast,Losonga,Ruvuma,10,3,Mbinga,Mbinga Urban,60,NaN,GeoData Consultants Ltd,Water Board,BRUDER,True,2000,gravity,gravity,gravity,water board,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


In [50]:
#7 features with Nans
test_features.isna().sum().sort_values().tail(7)

subvillage             94
permit                663
funder                783
public_meeting        785
installer             788
scheme_management     939
scheme_name          6839
dtype: int64

In [51]:
test_features.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,14358.000000,14358.000000,14358.0000,14358.000000,1.435800e+04,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000
mean,37232.859799,324.219996,653.6363,34.082414,-5.697584e+00,0.408971,15.156359,5.713052,187.055439,1298.251985
std,21382.890432,2533.367778,688.2721,6.564449,2.947444e+00,8.231859,17.387588,9.794304,476.065978,952.551852
min,10.000000,0.000000,-57.0000,0.000000,-1.156459e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18765.500000,0.000000,0.0000,33.062317,-8.453125e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37442.000000,0.000000,346.0000,34.898976,-5.087905e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55909.250000,25.000000,1306.0000,37.221606,-3.314240e+00,0.000000,17.000000,5.000000,230.000000,2004.000000
max,74249.000000,200000.000000,2777.0000,40.325016,-2.000000e-08,669.000000,99.000000,80.000000,11469.000000,2013.000000


In [52]:
#many features with high cardinality
test_features.describe(exclude='number')

,date_recorded,funder,installer,wpt_name,basin,subvillage,region,lga,ward,public_meeting,recorded_by,scheme_management,scheme_name,permit,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
count,14358,13575,13570,14358,14358,14264,14358,14358,14358,13573,14358,13419,7519,13695,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358
unique,331,960,1075,10615,9,8253,21,124,1934,2,1,11,1772,2,17,13,7,12,5,7,7,8,6,5,5,10,7,3,7,6
top,2011-03-16,Government Of Tanzania,DWE,none,Lake Victoria,Shuleni,Shinyanga,Njombe,Igosi,True,GeoData Consultants Ltd,VWC,Borehole,True,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,communal standpipe,communal standpipe
freq,137,2117,4162,822,2535,136,1258,611,79,12308,14358,8807,158,9442,6168,6168,6168,9780,12639,6098,6098,12237,12237,7997,7997,4211,4211,11127,6790,8260


In [53]:
#exploring features
test_features['permit'].value_counts(normalize=True)

True     0.689449
False    0.310551
Name: permit, dtype: float64

In [54]:
test_features.shape, train_features.shape, train_labels.shape

((14358, 40), (59400, 40), (59400, 2))

In [55]:
#majority class baseline accuracy score ~ 54% 
#our model should perform better then our naive basemodel 
train_labels['status_group'].value_counts(normalize=True)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

In [56]:
#since this is a classification problem, we will be using tree based models such as XGBoost and Random Forest
#overfitting could be an issue here, as 50% of the test set is held out in the private leaderboard 
#we will subdivide our training data set onto training and validation sets, we will use cross-validation 
#with independent test set, however we will be testing first on our new Kaggle subdivided validation set 

In [57]:
from sklearn.model_selection import train_test_split
X_train, X_train_val, y_train, y_train_val  = train_test_split(train_features, train_labels, test_size=0.4)

In [58]:
 X_train.shape, X_train_val.shape, y_train.shape, y_train_val.shape

((35640, 40), (23760, 40), (35640, 2), (23760, 2))

In [59]:
def wrangle(X):
    X = X.copy()
    
    #drop some columns 
    X = X.drop(columns='id') #id is random 
    X = X.drop(columns='date_recorded') #we don't care about the recorded row date 
    X = X.drop(columns='longitude') #does not add relevant data
    X = X.drop(columns='latitude') #does not add relevant data 
    X = X.drop(columns='num_private') # data dictionary has no description for this feature
    X = X.drop(columns='public_meeting') # data dictionary provides only "true/false"
    X = X.drop(columns='extraction_type_group') # # not duplicate but bery similar thus contains high variance of source feature
    X = X.drop(columns='extraction_type_class')
    X = X.drop(columns='payment_type') # duplicate feature
    X = X.drop(columns='quantity_group') # duplicate feature
    X = X.drop(columns='source_type') # not duplicate but bery similar thus contains high variance of source feature
    X = X.drop(columns='waterpoint_type_group') # not duplicate but very similar thus contains high variance of source feature
    X = X.drop(columns='region') #duplicates
    X = X.drop(columns='quality_group') #duplicates 
    X = X.drop(columns='source_class') #duplicates
    X = X.drop(columns='management_group') #duplicates
  
    #drop high cardinality categoricals
    X = X.drop(columns='funder')
    X = X.drop(columns='installer')
    X = X.drop(columns='wpt_name')
    X = X.drop(columns='subvillage')
    X = X.drop(columns='ward')
    X = X.drop(columns='scheme_name')
               
    
    #drop Nans from 'scheme_management' and 'permit' feature 
    X['scheme_management'].dropna()
    X['permit'].dropna()
    
    return X 


X_train = wrangle(X_train)
X_train_val = wrangle(X_train_val)  
X_test = wrangle(test_features)

In [60]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [61]:
X_train.head()

,amount_tsh,gps_height,basin,region_code,district_code,lga,population,recorded_by,scheme_management,permit,construction_year,extraction_type,management,payment,water_quality,quantity,source,waterpoint_type
30790,50.0,110,Ruvuma / Southern Coast,80,23,Lindi Rural,1,GeoData Consultants Ltd,VWC,False,1979,ksb,vwc,pay per bucket,soft,enough,spring,communal standpipe
49882,10.0,1145,Pangani,3,5,Hai,35,GeoData Consultants Ltd,Water Board,True,1999,gravity,water board,pay per bucket,soft,enough,spring,communal standpipe
21930,0.0,0,Lake Victoria,19,4,Kwimba,0,GeoData Consultants Ltd,VWC,True,0,other,vwc,never pay,soft,enough,shallow well,hand pump
48514,0.0,1187,Lake Rukwa,15,1,Mpanda,500,GeoData Consultants Ltd,Water authority,True,1989,other,water authority,never pay,soft,insufficient,hand dtw,other
40661,0.0,0,Lake Victoria,18,2,Bukoba Rural,0,GeoData Consultants Ltd,VWC,True,0,gravity,vwc,never pay,soft,insufficient,spring,improved spring


In [62]:
y_train.head()

,id,status_group
30790,62850,non functional
49882,36652,non functional
21930,34604,functional
48514,23912,non functional
40661,63647,functional needs repair


In [63]:
#1 is functional, 2 functional needs repair, 3 non functional
import category_encoders as ce
encoder = ce.OrdinalEncoder()
y_train = encoder.fit_transform(y_train)
y_train.head()

,id,status_group
30790,62850,1
49882,36652,1
21930,34604,2
48514,23912,1
40661,63647,3


In [64]:
import category_encoders as ce
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier

encoder = ce.OrdinalEncoder()
X_train = encoder.fit_transform(X_train)
encoder = ce.OrdinalEncoder()
y_train = encoder.fit_transform(y_train['status_group'])


param_distributions = {
    'n_estimators': randint(10, 50),
    'max_depth': randint(1, 3),
}

search = RandomizedSearchCV(
    estimator=XGBClassifier(), 
    param_distributions=param_distributions, 
    n_iter=2, 
    n_jobs=-1, 
    cv=2, 
    verbose=10, 
    return_train_score=True, 
    random_state=42
)

search.fit(X_train, y_train)

Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:   11.1s remaining:   11.1s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   13.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   13.4s finished


RandomizedSearchCV(cv=2, error_score='raise-deprecating',
          estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
          fit_params=None, iid='warn', n_iter=2, n_jobs=-1,
          param_distributions={'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x122882fd0>, 'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1a25047290>},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score=True, scoring=None, verbose=10)

In [65]:
# from sklearn.model_selection import cross_val_score

# accuracy = cross_val_score(search, X_train_val, y_train_val, scoring='accuracy', cv=5).mean()

In [66]:
#accuracy score for CV set is ok

search.best_score_

0.6944725028058362

In [67]:
best = search.best_estimator_
best

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=1, min_child_weight=1, missing=None, n_estimators=38,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [68]:
# import matplotlib.pyplot as plt

# figsize = (5, 15)
# plt.figure(figsize=figsize)
# importances = pd.Series(best.feature_importances_,X_train.columns).sort_values()
# importances.plot.barh()

In [69]:
# import eli5
# from eli5.sklearn import PermutationImportance

# permuter = PermutationImportance(best, cv='prefit', scoring='accuracy', 
#                                  n_iter=2, random_state=42)

# permuter.fit(X_train_val, y_train_val.values)

In [70]:
encoder = ce.OrdinalEncoder()
X_train_val = encoder.fit_transform(X_train_val)
X_train_val.head()

,amount_tsh,gps_height,basin,region_code,district_code,lga,population,recorded_by,scheme_management,permit,construction_year,extraction_type,management,payment,water_quality,quantity,source,waterpoint_type
25683,50.0,244,1,99,1,1,200,1,1,1,2012,1,1,1,1,1,1,1
43305,0.0,1323,2,2,7,2,124,1,2,1,1982,2,2,2,2,2,2,2
8098,0.0,0,3,17,3,3,0,1,3,1,0,3,3,3,1,3,3,3
12207,0.0,0,4,12,7,4,0,1,1,1,0,2,1,4,1,3,4,2
53871,0.0,1704,2,2,7,2,136,1,4,1,1989,2,4,2,1,3,2,2


In [71]:
encoder = ce.OrdinalEncoder()
y_train_val = encoder.fit_transform(y_train_val['status_group'])
y_train_val.head()

,status_group
25683,1
43305,1
8098,1
12207,1
53871,1


In [72]:
y_train_val['status_group'].value_counts()

1    12896
2     9118
3     1746
Name: status_group, dtype: int64

In [73]:
y_pred = best.predict(X_train_val)
y_pred = pd.DataFrame(columns=['status_group'], data=y_pred)
y_pred.head()

,status_group
0,2
1,2
2,1
3,2
4,2


In [74]:
y_pred['status_group'].value_counts()

2    12731
1    11029
Name: status_group, dtype: int64

In [76]:
#calculate accuracy for a-la-Kaggle test score 
#accuracy score is horrible
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_train_val, y_pred)
accuracy

0.33484848484848484

In [ ]:
#please ignore everything from this point forward







In [ ]:
#calculate predictions for submission
encoder = ce.OrdinalEncoder()
X_test = encoder.fit_transform(X_test)
y_pred_submission = best.predict(X_test)
a = pd.DataFrame(columns={'status_group'}, data=y_pred_submission)
a['status_group'].value_counts()

In [79]:
sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission['status_group'] = y_pred_submission
submission = submission.replace({1: 'functional', 3: 'functional needs repair', 2: 'non functional'})
submission.to_csv('praywithme.csv', index=False)

In [80]:
submission['status_group'].value_counts(normalize=True)

functional        0.717927
non functional    0.282073
Name: status_group, dtype: float64

In [81]:
from sklearn.ensemble import RandomForestClassifier
encoder = ce.OrdinalEncoder()
X_train = encoder.fit_transform(X_train)
model = RandomForestClassifier(max_depth=None, n_estimators=200, n_jobs=-1, random_state=42)
model.fit(X_train, y_train)

/Users/TomasFox/anaconda3/lib/python2.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [82]:
from sklearn.model_selection import cross_val_score

accuracy = cross_val_score(model, X_train_val, y_train_val, scoring='accuracy', cv=4).mean()
accuracy

0.7760933202987598

In [83]:
encoder = ce.OrdinalEncoder()
X_test = encoder.fit_transform(X_train_val)
y_pred_cv = model.predict(X_train_val)
a = pd.DataFrame(columns={'status_group'}, data=y_pred_cv)
a['status_group'].value_counts()

1    12833
2    10921
3        6
Name: status_group, dtype: int64

In [84]:
accuracy_score(y_train_val, y_pred_cv)

0.32752525252525255

In [85]:
#calculate predictions for submission
encoder = ce.OrdinalEncoder()
X_test = encoder.fit_transform(X_test)
y_pred_submission = model.predict(X_test)
a = pd.DataFrame(columns={'status_group'}, data=y_pred_submission)
a['status_group'].value_counts()

1    12833
2    10921
3        6
Name: status_group, dtype: int64

In [86]:
X_test.shape

(23760, 18)

In [87]:
sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission['status_group'] = y_pred_submission
submission = submission.replace({1: 'functional', 3: 'functional needs repair', 2: 'non functional'})
submission.to_csv('randomforest.csv', index=False)

ValueError: Length of values does not match length of index

In [ ]:
submission['status_group'].value_counts(normalize=True)

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

permuter = PermutationImportance(model, cv='prefit', scoring='accuracy', 
                                 n_iter=5, random_state=42)

permuter.fit(X_train_val, y_train_val.values)

In [ ]:
feature_names = X_train_val.columns.tolist()
eli5.show_weights(permuter, top=None, feature_names=feature_names)